In [1]:
import requests
import json

from auth import API_KEY

## 🖼️ Step 0: Which data do we want?

In [2]:
# Define paths to the images
paths = [
    'PATH_TO_IMAGE_1',
    'PATH_TO_IMAGE_2',
]

# Create a dictionary of filenames (key) --> filepaths (value) pairs
name_p = {}
for p in paths:
    name = p.split('/')[-1]
    name_p[name] = p

name_p

{'Cat_4.webp': 'data/Cat_4.webp', 'Cat_5.webp': 'data/Cat_5.webp'}

## 🗃️ Step 1: Register the data

In [3]:
team_slug = "YOUR_TEAM_SLUG"
dataset_slug = "YOUR_DATASET_SLUG"

url = f"https://darwin.v7labs.com/api/v2/teams/{team_slug}/items/register_upload"

# Create the payload for data registration
slots = []
for item in name_p:
    slots.append({
        "slots": [
        {
            "as_frames": False,
            "file_name": item,
            "slot_name": '0'
        }
        ],
        "path": "/",
        "name": item
        }
        )
    
payload = {
    "items": slots,
    "dataset_slug": dataset_slug
}

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"ApiKey {API_KEY}"
}

response = requests.post(url, json=payload, headers=headers)

print(response.text)

{"blocked_items":[],"items":[{"id":"018755eb-3b51-c16a-12d7-301755ea6c5e","name":"Cat_5.webp","path":"/","slots":[{"as_frames":false,"extract_views":false,"file_name":"Cat_5.webp","slot_name":"0","type":"image","upload_id":"999bb002-7945-4b3a-a8b5-9950c5a19013"}]},{"id":"018755eb-3b51-c961-1e0d-eb6bbdd0fb95","name":"Cat_4.webp","path":"/","slots":[{"as_frames":false,"extract_views":false,"file_name":"Cat_4.webp","slot_name":"0","type":"image","upload_id":"548466c4-0b89-4a7c-b628-5c12d648ca87"}]}]}


In [4]:
# Get upload IDs
upload_uid_p = {}
for item in json.loads(response.text)['items']:
    upload_uid_p[item['slots'][0]['upload_id']] = name_p[item['name']]

In [5]:
upload_uid_p

{'999bb002-7945-4b3a-a8b5-9950c5a19013': 'data/Cat_5.webp',
 '548466c4-0b89-4a7c-b628-5c12d648ca87': 'data/Cat_4.webp'}

## Steps 2-3-4: 🔏 Sign, 📤 Upload, and ✅ Confirm

In [6]:
# Perform steps 2, 3, & 4 for each item
for upload_id in upload_uid_p:

    # 2: Sign the upload, store the upload URL
    url = f"https://darwin.v7labs.com/api/v2/teams/{team_slug}/items/uploads/{upload_id}/sign"
    response = requests.get(url, headers=headers)
    upload_url = json.loads(response.text)['upload_url']

    # 3: Upload the file using the URL and local filepath
    with open(upload_uid_p[upload_id], 'rb') as f:
        data = f.read()
    response = requests.put(url=upload_url, data=data)
    print(response, response.text)
    
    # 4: Confirm the upload
    url = f'https://darwin.v7labs.com/api/v2/teams/{team_slug}/items/uploads/{upload_id}/confirm'
    response = requests.post(url, headers=headers)
    print(response, response.text)

<Response [200]> 
<Response [200]> {}
<Response [200]> 
<Response [200]> {}
